In [1]:
import pandas as pd
path_train = "/Users/gunnvantsaini/Documents/Data/Kaggle/News/BBC News Train.csv"
path_test = "/Users/gunnvantsaini/Documents/Data/Kaggle/News/BBC News Test.csv"

In [4]:
train = pd.read_csv(path_train)
test = pd.read_csv(path_test)

In [5]:
train.head(2)

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business


In [6]:
## We will create a classifier using embedding layer architecture
X=train['Text']
y=train['Category']

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [8]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=200)

In [9]:
enc=LabelEncoder()

In [10]:
y_train=enc.fit_transform(y_train)

In [11]:
enc.classes_

array(['business', 'entertainment', 'politics', 'sport', 'tech'],
      dtype=object)

In [12]:
y_train

array([4, 3, 0, ..., 3, 1, 1])

In [14]:
import tensorflow as tf

In [16]:
#### tf2.2+ ####
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [17]:
seq_len=16

In [18]:
vectorizer=TextVectorization(max_tokens=30000,output_sequence_length=seq_len) ### tfidf object

Metal device set to: Apple M1


2022-01-27 10:40:18.457948: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-27 10:40:18.458891: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [19]:
text_ds=tf.data.Dataset.from_tensor_slices(X_train).batch(64)

In [20]:
text_ds

<BatchDataset shapes: (None,), types: tf.string>

In [21]:
vectorizer.adapt(text_ds) ### fit_transform

2022-01-27 10:40:45.600609: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-01-27 10:40:45.601068: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [22]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'to', 'of']

In [23]:
len(vectorizer.get_vocabulary())

24865

In [24]:
sent='the cat ate the rat dfnasdfn'

In [26]:
import numpy as np
np.array([[sent]])

array([['the cat ate the rat dfnasdfn']], dtype='<U28')

In [27]:
vectorizer(np.array([[sent]])) ### words will have an offset of 2
### 0->padding
### 1-> oov words

<tf.Tensor: shape=(1, 16), dtype=int64, numpy=
array([[    2,  7541, 23495,     2, 12131,     1,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0]])>

In [30]:
from tensorflow import keras

In [31]:
from tensorflow.keras import layers

In [37]:
inputs = keras.Input(shape=(1,),dtype='string')
x = vectorizer(inputs)
x = layers.Embedding(input_dim=24865,output_dim=64)(x) ### [obj1,obj2,...obj16] ### 50, 100,300
x = layers.Flatten()(x)
x = layers.Dense(64,activation='relu')(x)
x = layers.Dense(4,activation='softmax')(x)

In [38]:
model=tf.keras.Model(inputs,x)

In [39]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization (TextVect (None, 16)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 16, 64)            1591360   
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                65600     
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 260       
Total params: 1,657,220
Trainable params: 1,657,220
Non-trainable params: 0
_________________________________________________

In [40]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])

In [36]:
model.fit(X_train,y_train,batch_size=128,epochs=5,validation_split=0.10)

Epoch 1/5


2022-01-27 10:43:49.643334: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 1s 36ms/step - loss: 1.1276 - acc: 0.2761 - val_loss: 1.2173 - val_acc: 0.4167
Epoch 2/5
7/9 [======================>.......] - ETA: 0s - loss: 1.0386 - acc: 0.7344

2022-01-27 10:43:50.326112: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 0s 19ms/step - loss: 1.0402 - acc: 0.7341 - val_loss: 1.1792 - val_acc: 0.4417
Epoch 3/5
9/9 [==============================] - 0s 19ms/step - loss: 0.9145 - acc: 0.7976 - val_loss: 1.1035 - val_acc: 0.5583
Epoch 4/5
9/9 [==============================] - 0s 20ms/step - loss: 0.7241 - acc: 0.8116 - val_loss: 0.9888 - val_acc: 0.6500
Epoch 5/5
9/9 [==============================] - 0s 17ms/step - loss: 0.5072 - acc: 0.8162 - val_loss: 0.8729 - val_acc: 0.6583
